In [1]:
pip install nltk scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
print('hi')

hi


In [3]:
import nltk
from nltk import ngrams
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.util import bigrams
from nltk.util import trigrams
import cProfile
import pandas as pd

def profile_code():
    sentences = pd.read_csv("True_news.csv")
    sentences = sentences.drop(['title', 'subject','date'], axis=1)
    print(sentences)
    sentences = sentences['text']
    sentences = sentences[:100]
    
    def preprocess_sentences(sentences):
        preprocessed = []
        for sentence in sentences:
         
            tokens = nltk.word_tokenize(sentence.lower())
            preprocessed.append(tokens)
        return preprocessed

    
    preprocessed_sentences = preprocess_sentences(sentences)


    def train_and_evaluate_bigram_model(sentences):
     
        bigram_model = defaultdict(lambda: defaultdict(lambda: 0))
        vocabulary_size = len(set([token for sentence in sentences for token in sentence]))

        for sentence in sentences:
            for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
                bigram_model[w1][w2] += 1

  
        for w1 in bigram_model:
            for w2 in bigram_model[w1]:
                bigram_model[w1][w2] = (bigram_model[w1][w2] + 1) / (sum(bigram_model[w1].values()) + vocabulary_size)

 
        def is_sentence_correct(tokens):
            tokens = [token.lower() for token in tokens]
            prob = 1.0
            for w1, w2 in bigrams(tokens, pad_right=True, pad_left=True):
                prob *= bigram_model[w1][w2]
            return prob

        # Test accuracy
        correct_predictions = 0
        for sentence_tokens in sentences:
            prob = is_sentence_correct(sentence_tokens)
            if prob > 0.5:
                correct_predictions += 1

        accuracy = correct_predictions / len(sentences)
        return accuracy

 
    def train_and_evaluate_trigram_model(sentences):
        trigram_model = defaultdict(lambda: defaultdict(lambda: 0))
        vocabulary_size = len(set([token for sentence in sentences for token in sentence])) 

        for sentence in sentences:
            for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
                trigram_model[(w1, w2)][w3] += 1

      
        for w1_w2 in trigram_model:
            for w3 in trigram_model[w1_w2]:
                trigram_model[w1_w2][w3] = (trigram_model[w1_w2][w3] + 1) / (sum(trigram_model[w1_w2].values()) + vocabulary_size)


        def is_sentence_correct(tokens):
            tokens = [token.lower() for token in tokens]
            prob = 1.0
            for w1, w2, w3 in trigrams(tokens, pad_right=True, pad_left=True):
                prob *= trigram_model[(w1, w2)][w3]
            return prob

        # Test accuracy
        correct_predictions = 0
        for sentence_tokens in sentences:
            prob = is_sentence_correct(sentence_tokens)
            if prob > 0.5:  
                correct_predictions += 1

        accuracy = correct_predictions / len(sentences)
        return accuracy



    def train_and_evaluate_unigram_model(sentences):
    
        unigram_model = defaultdict(lambda: 0)
        vocabulary_size = len(set([token for sentence in sentences for token in sentence]))  
        total_tokens = 0

        for sentence in sentences:
            for token in sentence:
                unigram_model[token] += 1
                total_tokens += 1

    
        for token in unigram_model:
            unigram_model[token] = (unigram_model[token] + 1) / (total_tokens + vocabulary_size)

        def is_sentence_correct(tokens):
            prob = 1.0
            for token in tokens:
                prob *= unigram_model[token]
            return prob

        # Test accuracy
        correct_predictions = 0
        for sentence_tokens in sentences:
            prob = is_sentence_correct(sentence_tokens)
            if prob > 0.5:  
                correct_predictions += 1

        accuracy = correct_predictions / len(sentences)
        return accuracy

    def train_and_evaluate_quadragram_model(sentences):
        # Create quadragram model
        quadragram_model = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0))))
        vocabulary_size = len(set([token for sentence in sentences for token in sentence]))  # Size of the vocabulary
        total_quadragrams = 0

        for sentence in sentences:
            for w1, w2, w3, w4 in ngrams(sentence, 4, pad_right=True, pad_left=True):
                quadragram_model[w1][w2][w3][w4] += 1
                total_quadragrams += 1

        # Apply Laplace smoothing
        for w1 in quadragram_model:
            for w2 in quadragram_model[w1]:
                for w3 in quadragram_model[w1][w2]:
                    for w4 in quadragram_model[w1][w2][w3]:
                        quadragram_model[w1][w2][w3][w4] = (quadragram_model[w1][w2][w3][w4] + 1) / (total_quadragrams + vocabulary_size)

        # Function to predict sentence correctness using quadragram model
        def is_sentence_correct(tokens):
            prob = 1.0
            for w1, w2, w3, w4 in ngrams(tokens, 4, pad_right=True, pad_left=True):
                prob *= quadragram_model[w1][w2][w3][w4]
            return prob

        # Test accuracy
        correct_predictions = 0
        for sentence_tokens in sentences:
            prob = is_sentence_correct(sentence_tokens)
            if prob > 0.5:  # Adjust threshold as needed
                correct_predictions += 1

        accuracy = correct_predictions / len(sentences)
        return accuracy

    # Compare accuracies of different models
    unigram_accuracy = train_and_evaluate_unigram_model(preprocessed_sentences)
    quadragram_accuracy = train_and_evaluate_quadragram_model(preprocessed_sentences)
    bigram_accuracy = train_and_evaluate_bigram_model(preprocessed_sentences)
    trigram_accuracy = train_and_evaluate_trigram_model(preprocessed_sentences)

    print("Unigram Model Accuracy:", unigram_accuracy)
    print("Quadragram Model Accuracy:", quadragram_accuracy)
    print("Bigram Model Accuracy:", bigram_accuracy)
    print("Trigram Model Accuracy:", trigram_accuracy)

cProfile.run("profile_code()")


                                                    text
0      WASHINGTON (Reuters) - The head of a conservat...
1      WASHINGTON (Reuters) - Transgender people will...
2      WASHINGTON (Reuters) - The special counsel inv...
3      WASHINGTON (Reuters) - Trump campaign adviser ...
4      SEATTLE/WASHINGTON (Reuters) - President Donal...
...                                                  ...
21412  BRUSSELS (Reuters) - NATO allies on Tuesday we...
21413  LONDON (Reuters) - LexisNexis, a provider of l...
21414  MINSK (Reuters) - In the shadow of disused Sov...
21415  MOSCOW (Reuters) - Vatican Secretary of State ...
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...

[21417 rows x 1 columns]
Unigram Model Accuracy: 0.0
Quadragram Model Accuracy: 0.0
Bigram Model Accuracy: 0.0
Trigram Model Accuracy: 0.0
         752673 function calls (744354 primitive calls) in 1.603 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)


      297    0.000    0.000    0.000    0.000 generic.py:43(_check)
        3    0.000    0.000    0.000    0.000 generic.py:437(_validate_dtype)
        2    0.000    0.000    0.001    0.000 generic.py:4571(reindex)
        4    0.000    0.000    0.000    0.000 generic.py:4804(<genexpr>)
        1    0.000    0.000    0.000    0.000 generic.py:4848(_needs_reindex_multi)
        1    0.000    0.000    0.001    0.001 generic.py:4860(_reindex_with_indexers)
        2    0.000    0.000    0.000    0.000 generic.py:510(_construct_axes_from_arguments)
        2    0.000    0.000    0.000    0.000 generic.py:539(<dictcomp>)
       25    0.000    0.000    0.000    0.000 generic.py:542(_get_axis_number)
        8    0.000    0.000    0.000    0.000 generic.py:5435(__finalize__)
        3    0.000    0.000    0.000    0.000 generic.py:5466(<genexpr>)
        7    0.000    0.000    0.000    0.000 generic.py:5473(__getattr__)
       14    0.000    0.000    0.000    0.000 generic.py:5489(__setattr

     6157    0.001    0.000    0.001    0.000 {method 'join' of 'str' objects}
        3    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
       85    0.000    0.000    0.000    0.000 {method 'ljust' of 'str' objects}
   100710    0.012    0.000    0.012    0.000 {method 'lower' of 'str' objects}
      103    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
     7292    0.003    0.000    0.003    0.000 {method 'match' of 're.Pattern' objects}
        3    0.000    0.000    0.000    0.000 {method 'max' of 'numpy.ndarray' objects}
       50    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 {method 'pop' of 'list' objects}
        4    0.000    0.000    0.000    0.000 {method 'pop' of 'set' objects}
       15    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'read' of '_io.BufferedReader' objects}
     